In [ ]:
import pprint
import requests 
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor
import seaborn as sns
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Informe tècnic/cajamar/DF.csv')

In [ ]:
# Visualización del DataFrame final
df

,TIMESTAMP,ID,READING,DELTA
0,2019-02-01,0,7963359.0,243.0
1,2019-02-02,0,7969096.0,236.0
2,2019-02-03,0,7975263.0,335.0
3,2019-02-04,0,7982426.0,252.0
4,2019-02-05,0,7988972.0,220.0
...,...,...,...,...
890826,2019-08-03,2748,1627386.0,313.0
890827,2019-09-14,2748,1690044.0,8.0
890828,2019-08-06,2749,627196.0,0.0
890829,2019-10-11,2749,658022.0,2.0


In [ ]:
df['TIMESTAMP']=pd.to_datetime(df['TIMESTAMP'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890831 entries, 0 to 890830
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   TIMESTAMP  890831 non-null  datetime64[ns]
 1   ID         890831 non-null  int64         
 2   READING    890831 non-null  float64       
 3   DELTA      890831 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 27.2 MB


In [ ]:
def missing_elements(L):
    start, end = L[0], L[-1]
    return sorted(set(range(start, end + 1)).difference(L))
missing_elements(list(df["ID"].unique().astype(int)))

[2726, 2738, 2740, 2741, 2750, 2751, 2752, 2753, 2754, 2755]

In [ ]:
!pip install catboost
!pip install skforecast
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

from joblib import dump, load

     |████████████████████████████████| 76.1 MB 52 kB/s 
     |████████████████████████████████| 87 kB 5.0 MB/s 
     |████████████████████████████████| 9.8 MB 41.2 MB/s 
     |████████████████████████████████| 76 kB 4.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.63.0
    Uninstalling tqdm-4.63.0:
      Successfully uninstalled tqdm-4.63.0
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Funciones necesarias para el preprocesado final de cada id

def fill_na_0(df):
  return df.fillna(0)

def fill_na_custom(df):
  return df.interpolate(method='piecewise_polynomial', axis=0).ffill().bfill()

def filter_outliers(df):
  lof = LocalOutlierFactor(contamination=0.05, n_neighbors=7)
  indx = lof.fit_predict(df) 
  mask = indx == 1
  df_test = df[mask]
  return df_test

def fill_na_mean_adj(df):
   return (df.ffill()+df.bfill())/2


In [ ]:
def pp_train_4_pred(train,id):
  date_index = pd.date_range('2019-02-1', periods=365, freq='D')
  train.set_index('TIMESTAMP', inplace=True)
  train = train.asfreq('D')
  train=train.reindex(date_index)
  if(train['DELTA'].isna().sum()>=364):
    train = fill_na_0(train)
  else:
    train = fill_na_custom(train)
    train = fill_na_0(train)
    train = filter_outliers(train)
    train = train.asfreq('D')
    train=train.reindex(date_index)
    train = fill_na_mean_adj(train)
    train = fill_na_custom(train)
  return train

def pp_train_4_pred2(train,id,periods):
  date_index = pd.date_range('2019-02-1', periods=periods, freq='D')
  train.set_index('TIMESTAMP', inplace=True)
  train = train.asfreq('D')
  train=train.reindex(date_index)
  if(train['DELTA'].isna().sum()<364):
    for item,frame in train["DELTA"].iteritems():
      if(pd.isnull(frame)):
        train.loc[item]["ID"]=id
        loc = train.index.get_loc(item)
        train.loc[item]["READING"]=train.iloc[loc - 1]["READING"]
        for i in range(int(len(list(train.index))/14)):
          if((loc - i*7)>0 and (loc + i*7)<periods):
            if(not np.isnan(train.iloc[loc - i*7]["DELTA"]) and not np.isnan(train.iloc[loc + i*7]["DELTA"])):
              train.loc[item]["DELTA"]=(train.iloc[loc - i*7]["DELTA"]+train.iloc[loc + i*7]["DELTA"])/2
              break
        if(np.isnan(train.iloc[loc]["DELTA"])):
          mean=[]
          for i in range(8):
            if((loc - i*7)>0 and (loc + i*7)<periods):
              if(not np.isnan(train.iloc[loc - i*7]["DELTA"])):
                  mean.append(train.iloc[loc - i*7]["DELTA"])
              elif(not np.isnan(train.iloc[loc + i*7]["DELTA"])):
                  mean.append(train.iloc[loc - i*7]["DELTA"]) 
          if(len(mean)>0):
            train.loc[item]["DELTA"]=sum(mean)/len(mean)
    train = fill_na_custom(train)
    train = filter_outliers(train)
    train = train.asfreq('D')
    train = train.reindex(date_index)
    train = fill_na_mean_adj(train)
    train = fill_na_custom(train)
  else:
    train = fill_na_0(train)
  return train


In [ ]:
# Obtención del dataset de predicciones de febrero de 2020 para la partición 1

# NOTA: la ejecución de esta celda es bastante costosa.

df_output = pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
llista_id = df["ID"].unique()
for id in llista_id[:344]:
    print("----------------")
    print("ID:",id)
    train=df[df["ID"]==id]
    train = pp_train_4_pred2(train,id,365)
    forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=30,random_state=0),
                    lags      = 12,
                 )
    param_grid = {}
    lags_grid = {7,14,21,28,35,42,49,56}
    results_grid = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = train['DELTA'],
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = 5,
                                refit              = True,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(train)*0.5),
                                return_best        = True,
                                verbose            = False
                          )
    forecaster.fit(y = train["DELTA"])
    predictions =(forecaster.predict(steps=20))
    row = {"ID":id,"1":round(predictions.values[0],2),"2":round(predictions.values[1],2),"3":round(predictions.values[2],2),"4":round(predictions.values[3],2),"5":round(predictions.values[4],2),"6":round(predictions.values[5],2),"7":round(predictions.values[6],2),"Suma1":round(sum(predictions.values[0:7]),2),"Suma2":round(sum(predictions.values[7:14]))}
    df_output = df_output.append(row, ignore_index=True)

df_output.ID = df_output.ID.astype(int)
df_output.to_csv('Part1.txt', header=False, index=False, sep="|")

In [ ]:
# Obtención del dataset de predicciones de febrero de 2020 para la partición 2

# NOTA: la ejecución de esta celda es bastante costosa.

df_output = pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
llista_id = df["ID"].unique()
for id in llista_id[344:688]:
    print("----------------")
    print("ID:",id)
    train=df[df["ID"]==id]
    train = pp_train_4_pred2(train,id,365)
    forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=30,random_state=0),
                    lags      = 12,
                 )
    param_grid = {}
    lags_grid = {7,14,21,28,35,42,49,56}
    results_grid = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = train['DELTA'],
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = 5,
                                refit              = True,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(train)*0.5),
                                return_best        = True,
                                verbose            = False
                          )
    forecaster.fit(y = train["DELTA"])
    predictions =(forecaster.predict(steps=20))
    row = {"ID":id,"1":round(predictions.values[0],2),"2":round(predictions.values[1],2),"3":round(predictions.values[2],2),"4":round(predictions.values[3],2),"5":round(predictions.values[4],2),"6":round(predictions.values[5],2),"7":round(predictions.values[6],2),"Suma1":round(sum(predictions.values[0:7]),2),"Suma2":round(sum(predictions.values[7:14]))}
    df_output = df_output.append(row, ignore_index=True)

df_output.ID = df_output.ID.astype(int)
df_output.to_csv('Part2.txt', header=False, index=False, sep="|")

In [ ]:
# Obtención del dataset de predicciones de febrero de 2020 para la partición 3

# NOTA: la ejecución de esta celda es bastante costosa.

df_output = pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
llista_id = df["ID"].unique()
for id in llista_id[688:1032]:
    print("----------------")
    print("ID:",id)
    train=df[df["ID"]==id]
    train = pp_train_4_pred2(train,id,365)
    forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=30,random_state=0),
                    lags      = 12,
                 )
    param_grid = {}
    lags_grid = {7,14,21,28,35,42,49,56}
    results_grid = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = train['DELTA'],
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = 5,
                                refit              = True,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(train)*0.5),
                                return_best        = True,
                                verbose            = False
                          )
    forecaster.fit(y = train["DELTA"])
    predictions =(forecaster.predict(steps=20))
    row = {"ID":id,"1":round(predictions.values[0],2),"2":round(predictions.values[1],2),"3":round(predictions.values[2],2),"4":round(predictions.values[3],2),"5":round(predictions.values[4],2),"6":round(predictions.values[5],2),"7":round(predictions.values[6],2),"Suma1":round(sum(predictions.values[0:7]),2),"Suma2":round(sum(predictions.values[7:14]))}
    df_output = df_output.append(row, ignore_index=True)

df_output.ID = df_output.ID.astype(int)
df_output.to_csv('Part3.txt', header=False, index=False, sep="|")

In [ ]:
# Obtención del dataset de predicciones de febrero de 2020 para la partición 4

# NOTA: la ejecución de esta celda es bastante costosa.

df_output = pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
llista_id = df["ID"].unique()
for id in llista_id[1032:1376]:
    print("----------------")
    print("ID:",id)
    train=df[df["ID"]==id]
    train = pp_train_4_pred2(train,id,365)
    forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=30,random_state=0),
                    lags      = 12,
                 )
    param_grid = {}
    lags_grid = {7,14,21,28,35,42,49,56}
    results_grid = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = train['DELTA'],
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = 5,
                                refit              = True,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(train)*0.5),
                                return_best        = True,
                                verbose            = False
                          )
    forecaster.fit(y = train["DELTA"])
    predictions =(forecaster.predict(steps=20))
    row = {"ID":id,"1":round(predictions.values[0],2),"2":round(predictions.values[1],2),"3":round(predictions.values[2],2),"4":round(predictions.values[3],2),"5":round(predictions.values[4],2),"6":round(predictions.values[5],2),"7":round(predictions.values[6],2),"Suma1":round(sum(predictions.values[0:7]),2),"Suma2":round(sum(predictions.values[7:14]))}
    df_output = df_output.append(row, ignore_index=True)

df_output.ID = df_output.ID.astype(int)
df_output.to_csv('Part4.txt', header=False, index=False, sep="|")

In [ ]:
# Obtención del dataset de predicciones de febrero de 2020 para la partición 5

# NOTA: la ejecución de esta celda es bastante costosa.

df_output = pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
llista_id = df["ID"].unique()
for id in llista_id[1376:1720]:
    print("----------------")
    print("ID:",id)
    train=df[df["ID"]==id]
    train = pp_train_4_pred2(train,id,365)
    forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=30,random_state=0),
                    lags      = 12,
                 )
    param_grid = {}
    lags_grid = {7,14,21,28,35,42,49,56}
    results_grid = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = train['DELTA'],
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = 5,
                                refit              = True,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(train)*0.5),
                                return_best        = True,
                                verbose            = False
                          )
    forecaster.fit(y = train["DELTA"])
    predictions =(forecaster.predict(steps=20))
    row = {"ID":id,"1":round(predictions.values[0],2),"2":round(predictions.values[1],2),"3":round(predictions.values[2],2),"4":round(predictions.values[3],2),"5":round(predictions.values[4],2),"6":round(predictions.values[5],2),"7":round(predictions.values[6],2),"Suma1":round(sum(predictions.values[0:7]),2),"Suma2":round(sum(predictions.values[7:14]))}
    df_output = df_output.append(row, ignore_index=True)

df_output.ID = df_output.ID.astype(int)
df_output.to_csv('Part5.txt', header=False, index=False, sep="|")

In [ ]:
# Obtención del dataset de predicciones de febrero de 2020 para la partición 6

# NOTA: la ejecución de esta celda es bastante costosa.

df_output = pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
llista_id = df["ID"].unique()
for id in llista_id[1720:2064]:
    print("----------------")
    print("ID:",id)
    train=df[df["ID"]==id]
    train = pp_train_4_pred2(train,id,365)
    forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=30,random_state=0),
                    lags      = 12,
                 )
    param_grid = {}
    lags_grid = {7,14,21,28,35,42,49,56}
    results_grid = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = train['DELTA'],
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = 5,
                                refit              = True,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(train)*0.5),
                                return_best        = True,
                                verbose            = False
                          )
    forecaster.fit(y = train["DELTA"])
    predictions =(forecaster.predict(steps=20))
    row = {"ID":id,"1":round(predictions.values[0],2),"2":round(predictions.values[1],2),"3":round(predictions.values[2],2),"4":round(predictions.values[3],2),"5":round(predictions.values[4],2),"6":round(predictions.values[5],2),"7":round(predictions.values[6],2),"Suma1":round(sum(predictions.values[0:7]),2),"Suma2":round(sum(predictions.values[7:14]))}
    df_output = df_output.append(row, ignore_index=True)

df_output.ID = df_output.ID.astype(int)
df_output.to_csv('Part6.txt', header=False, index=False, sep="|")

In [ ]:
# Obtención del dataset de predicciones de febrero de 2020 para la partición 7

# NOTA: la ejecución de esta celda es bastante costosa.

df_output = pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
llista_id = df["ID"].unique()
for id in llista_id[2064:2408]:
    print("----------------")
    print("ID:",id)
    train=df[df["ID"]==id]
    train = pp_train_4_pred2(train,id,365)
    forecaster = ForecasterAutoreg(
                    regressor = make_pipeline(StandardScaler(), Ridge()),
                    lags      = 12,
                 )
    param_grid = {}
    lags_grid = {7,14,21,28,35,42,49,56}
    results_grid = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = train['DELTA'],
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = 5,
                                refit              = True,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(train)*0.5),
                                return_best        = True,
                                verbose            = False
                          )
    forecaster.fit(y = train["DELTA"])
    predictions =(forecaster.predict(steps=20))
    row = {"ID":id,"1":round(predictions.values[0],2),"2":round(predictions.values[1],2),"3":round(predictions.values[2],2),"4":round(predictions.values[3],2),"5":round(predictions.values[4],2),"6":round(predictions.values[5],2),"7":round(predictions.values[6],2),"Suma1":round(sum(predictions.values[0:7]),2),"Suma2":round(sum(predictions.values[7:14]))}
    df_output = df_output.append(row, ignore_index=True)

df_output.ID = df_output.ID.astype(int)
df_output.to_csv('Part7.txt', header=False, index=False, sep="|")

In [ ]:
# Obtención del dataset de predicciones de febrero de 2020 para la partición 8

# NOTA: la ejecución de esta celda es bastante costosa.

df_output = pd.DataFrame()
import warnings
warnings.filterwarnings("ignore")
llista_id = df["ID"].unique()
for id in llista_id[2408:]:
    print("----------------")
    print("ID:",id)
    train=df[df["ID"]==id]
    train = pp_train_4_pred2(train,id,365)
    forecaster = ForecasterAutoreg(
                    regressor = make_pipeline(StandardScaler(), Ridge()),
                    lags      = 12,
                 )
    param_grid = {}
    lags_grid = {7,14,21,28,35,42,49,56}
    results_grid = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = train['DELTA'],
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = 5,
                                refit              = True,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(train)*0.5),
                                return_best        = True,
                                verbose            = False
                          )
    forecaster.fit(y = train["DELTA"])
    predictions =(forecaster.predict(steps=20))
    row = {"ID":id,"1":round(predictions.values[0],2),"2":round(predictions.values[1],2),"3":round(predictions.values[2],2),"4":round(predictions.values[3],2),"5":round(predictions.values[4],2),"6":round(predictions.values[5],2),"7":round(predictions.values[6],2),"Suma1":round(sum(predictions.values[0:7]),2),"Suma2":round(sum(predictions.values[7:14]))}
    df_output = df_output.append(row, ignore_index=True)

df_output.ID = df_output.ID.astype(int)
df_output.to_csv('Part8.txt', header=False, index=False, sep="|")

In [ ]:
#Juntamos las diferentes predicciones en un único fichero
df1 = pd.read_csv('/content/Part1.txt', header=None, sep='|')
df2 = pd.read_csv('/content/Part2.txt', header= None, sep='|')
df3 = pd.read_csv('/content/Part3.txt', header= None, sep='|')
df4 = pd.read_csv('/content/Part4.txt', header= None, sep='|')
df5 = pd.read_csv('/content/Part5.txt', header= None, sep='|')
df6 = pd.read_csv('/content/Part6.txt', header= None, sep='|')
df7 = pd.read_csv('/content/Part7.txt', header= None, sep='|')
df8 = pd.read_csv('/content/Part8.txt', header= None, sep='|')

frames = [df1, df2, df3, df4, df5, df6, df7,df8]

df_final = pd.concat(frames)
df_final

In [ ]:
df_final.to_csv('OvoCAD.txt', header=False, index=False, sep="|")